In [39]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

# changed animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AACoutcomes import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# updated with your username and password and CRUD Python module name

username = "aacuser"
password = "password1"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Added in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Tristin Watson'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'8%', 'width':'8%'}),
    html.Hr(),
    html.Div([
        html.Br(),
        html.Label(['Filter Options:'], style={'font-weight': 'bold'}),
        dcc.RadioItems(id='radio-item',
                       options=[{'label': 'Water Rescue', 'value': 'Water Rescue'},
                                {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
                                {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
                                {'label': 'Reset', 'value': 'Reset'},
                               ],
                      style={'width':'50%'}
                      ),
        ]),
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        # Set up the features for interactive data table to make it user-friendly for the client
        style_table={'maxWidth':'1500px'},
        editable=True,
        style_cell={"fontFamily":"Arial","size":10, "textAlign":"left"}
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

"""@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)"""

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

"""@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    fig = px.pie(dff['breed'], 
                 values='breed', 
                 names='Breeds', 
                 title='Grazioso Salvare',
                 width=500,
                 height=500,
                 hole=0.5)
    fig.update_traces()
    return [fig]"""

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
    

app